# Market Predictor

## Loading data

In [1]:
import numpy as np
import pandas as pd


df = pd.read_csv('csv/000997.csv')
df.head()

,Date,Open,Close,ChgVol,Chg,Low,High,Vol,Amt,TORate
0,2017-02-20,17.83,18.00,0.16,0.90,17.70,18.01,65991,11806.87,0.71
1,2017-02-17,18.10,17.84,-0.26,-1.44,17.80,18.17,95004,17051.83,1.03
2,2017-02-16,18.28,18.10,0.11,0.61,18.01,18.28,60777,10991.66,0.66
3,2017-02-15,18.30,17.99,-0.31,-1.69,17.93,18.37,102305,18599.72,1.11
4,2017-02-14,18.25,18.30,0.04,0.22,18.20,18.36,63664,11634.16,0.69


## Cleaning the data

### Calculating amplitude

In [2]:
df['Amp'] = df.apply(lambda row: (row['High'] - row['Low']) / row['Open'] * 100 ,axis=1)

### Droping redundant columns

In [3]:
clean_df = df.iloc[::-1].loc[:,['Chg', 'TORate','Amp']][1:]
clean_df[0:].reset_index(drop=True).head()

,Chg,TORate,Amp
0,0.40,7.47,2.840909
1,3.08,8.97,5.472496
2,-0.16,6.69,3.217889
3,-0.71,3.80,1.107726
4,-1.66,3.25,1.722222


### Creating dataframe of training samples

In [7]:
df_list = []

for i in range(10):
    df_list.append(clean_df[i:].reset_index(drop=True))

df_list.append(clean_df[['Chg']][10:].reset_index(drop=True))

sample_df = pd.concat(df_list, axis=1, join='inner', ignore_index=True)
sample_df.loc[:,30] = sample_df.apply(lambda row: 1 if row.loc[30] > 0 else 0, axis=1)
sample_df.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.40,7.47,2.840909,3.08,8.97,5.472496,-0.16,6.69,3.217889,-0.71,...,-0.86,1.26,2.082755,-0.31,1.72,1.098592,-1.15,2.51,1.380282,0
1,3.08,8.97,5.472496,-0.16,6.69,3.217889,-0.71,3.80,1.107726,-1.66,...,-0.31,1.72,1.098592,-1.15,2.51,1.380282,-2.22,2.07,2.724177,0
2,-0.16,6.69,3.217889,-0.71,3.80,1.107726,-1.66,3.25,1.722222,2.05,...,-1.15,2.51,1.380282,-2.22,2.07,2.724177,-2.04,3.58,4.903678,1
3,-0.71,3.80,1.107726,-1.66,3.25,1.722222,2.05,2.09,2.760563,-0.77,...,-2.22,2.07,2.724177,-2.04,3.58,4.903678,5.34,2.38,8.985765,1
4,-1.66,3.25,1.722222,2.05,2.09,2.760563,-0.77,1.67,1.405733,-0.86,...,-2.04,3.58,4.903678,5.34,2.38,8.985765,2.23,4.57,3.954802,0


## Building the deep neuron network

### creating the training sets

In [5]:
trainX = sample_df.loc[:, 0:29].values
trainX = np.array(trainX, ndmin=2)
trainX.shape

(3789, 30)

In [6]:
trainY = sample_df[30].values
trainY = np.array(trainY, ndmin=2).T
trainY.shape

(3789, 1)

### Building Network

In [10]:
import tensorflow as tf
import tflearn

tf.reset_default_graph()
tflearn.init_graph(num_cores=8, gpu_memory_fraction=0.5)

net = tflearn.input_data(shape=[None, trainX.shape[1]])
net = tflearn.fully_connected(net, 100, activation='relu')
net = tflearn.dropout(net, 0.5)
net = tflearn.fully_connected(net, 20, activation='relu')
net = tflearn.fully_connected(net, 1, activation='sigmoid')
net = tflearn.regression(net, optimizer='adagrad', learning_rate=0.01, loss=tflearn.objectives.mean_square)

model = tflearn.DNN(net, tensorboard_dir='./tflearn_logs/', tensorboard_verbose=3)

### Training Network

In [ ]:
model.fit(trainX, trainY, validation_set=0.1, show_metric=True, shuffle=True, batch_size=None, n_epoch=1000)

Training Step: 40396  | total loss: 0.22566 | time: 0.196s
| AdaGrad | epoch: 749 | loss: 0.22566 - binary_acc: 0.5072 -- iter: 0256/3410


### Saving model

In [29]:
model.save('my_model.tflearn')

INFO:tensorflow:D:\workspace\machine_learning\market-miner\my_model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


### Loading model

In [30]:
model.load('my_model.tflearn')

## Predicting market data

In [58]:
df_list = []

for i in range(10):
    df_list.append(clean_df[i:].reset_index(drop=True))

df_list.append(clean_df[['Chg']][10:].reset_index(drop=True))

predict_df = pd.concat(df_list, axis=1, join='outer', ignore_index=True)
predict_df = predict_df[::-1]
predict_df = predict_df.loc[[len(predict_df) - 10]]
predict_df

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
3789,2017-02-07,-0.49,0.62,1.092299,2017-02-08,0.55,0.89,1.975851,2017-02-09,0.33,...,1.477024,2017-02-17,-1.44,1.03,2.044199,2017-02-20,0.9,0.71,1.738643,NaN


In [55]:
X = predict_df.loc[:, 0:29].values
X = np.array(X, ndmin=2)
X.shape

(1, 30)

In [56]:
model.predict(X)

[[0.08913671970367432]]